### Modeling

In [116]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import Lasso, LassoCV, Ridge, RidgeCV, LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, AdaBoostRegressor, VotingRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint as sp_randint
from scipy.stats import uniform as sp_uniform
import lightgbm as lgb
from sklearn.feature_selection import SelectFromModel
pd.pandas.set_option('display.max_columns', None)
from mlxtend.feature_selection import SequentialFeatureSelector as sfs
from sklearn.metrics import mean_squared_error

In [117]:
X_train = pd.read_csv('X_train_dum_lasso.csv')
X_test = pd.read_csv('X_test_dum_lasso.csv')
display(X_train.head())
print(X_train.shape)
display(X_test.head())
print(X_test.shape)

,MSSubClass,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,1stFlrSF,2ndFlrSF,BsmtFullBath,FullBath,HalfBath,TotRmsAbvGrd,Fireplaces,GarageYrBlt,GarageCars,WoodDeckSF,ScreenPorch,MoSold,YrSold,MSZoning_RL,MSZoning_Rare_val,LotShape_IR2,LandContour_HLS,LandContour_Lvl,LotConfig_CulDSac,LotConfig_FR2,LandSlope_Mod,Neighborhood_BrkSide,Neighborhood_Crawfor,Neighborhood_Edwards,Neighborhood_Gilbert,Neighborhood_IDOTRR,Neighborhood_Mitchel,Neighborhood_NAmes,Neighborhood_NWAmes,Neighborhood_NoRidge,Neighborhood_NridgHt,Neighborhood_OldTown,Neighborhood_Sawyer,Neighborhood_Somerst,Neighborhood_StoneBr,Condition1_Norm,Condition2_Rare_val,BldgType_Duplex,BldgType_TwnhsE,HouseStyle_1Story,HouseStyle_2Story,RoofStyle_Hip,RoofMatl_Rare_val,Exterior1st_BrkFace,Exterior1st_CemntBd,Exterior1st_HdBoard,Exterior1st_Wd Sdng,Exterior2nd_Plywood,Exterior2nd_Stucco,Exterior2nd_VinylSd,MasVnrType_BrkFace,ExterQual_Gd,ExterQual_TA,Foundation_CBlock,Foundation_PConc,BsmtQual_Fa,BsmtQual_Gd,BsmtQual_TA,BsmtCond_Missing,BsmtCond_TA,BsmtExposure_Gd,BsmtExposure_Missing,BsmtExposure_No,BsmtFinType1_BLQ,BsmtFinType1_GLQ,BsmtFinType1_LwQ,BsmtFinType1_Unf,BsmtFinType2_Unf,HeatingQC_Gd,HeatingQC_TA,CentralAir_Y,KitchenQual_Fa,KitchenQual_Gd,KitchenQual_TA,Functional_Typ,GarageType_Missing,GarageType_Rare_val,GarageFinish_Missing,GarageFinish_RFn,GarageFinish_Unf,GarageQual_Missing,GarageQual_Rare_val,GarageCond_Missing,PavedDrive_Y,Fence_MnPrv,SaleType_New,SaleType_WD,SaleCondition_Normal
0,0.235294,0.366344,0.666667,0.500,0.036765,0.098361,0.12250,0.356155,0.413559,0.333333,0.666667,0.5,0.500000,0.000000,0.046729,0.50,0.000000,0.0,0.090909,0.50,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0
1,0.000000,0.391317,0.555556,0.875,0.227941,0.524590,0.00000,0.503056,0.000000,0.000000,0.666667,0.0,0.333333,0.333333,0.289720,0.50,0.347725,0.0,0.363636,0.25,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0
2,0.235294,0.422359,0.666667,0.500,0.051471,0.114754,0.10125,0.383441,0.419370,0.333333,0.666667,0.5,0.333333,0.333333,0.065421,0.50,0.000000,0.0,0.727273,0.50,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0
3,0.294118,0.390295,0.666667,0.500,0.669118,0.606557,0.00000,0.399941,0.366102,0.333333,0.333333,0.0,0.416667,0.333333,0.074766,0.75,0.000000,0.0,0.090909,0.00,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
4,0.235294,0.468761,0.777778,0.500,0.058824,0.147541,0.21875,0.466237,0.509927,0.333333,0.666667,0.5,0.583333,0.333333,0.074766,0.75,0.224037,0.0,1.000000,0.50,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0


(1460, 95)


,MSSubClass,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,1stFlrSF,2ndFlrSF,BsmtFullBath,FullBath,HalfBath,TotRmsAbvGrd,Fireplaces,GarageYrBlt,GarageCars,WoodDeckSF,ScreenPorch,MoSold,YrSold,MSZoning_RL,MSZoning_Rare_val,LotShape_IR2,LandContour_HLS,LandContour_Lvl,LotConfig_CulDSac,LotConfig_FR2,LandSlope_Mod,Neighborhood_BrkSide,Neighborhood_Crawfor,Neighborhood_Edwards,Neighborhood_Gilbert,Neighborhood_IDOTRR,Neighborhood_Mitchel,Neighborhood_NAmes,Neighborhood_NWAmes,Neighborhood_NoRidge,Neighborhood_NridgHt,Neighborhood_OldTown,Neighborhood_Sawyer,Neighborhood_Somerst,Neighborhood_StoneBr,Condition1_Norm,Condition2_Rare_val,BldgType_Duplex,BldgType_TwnhsE,HouseStyle_1Story,HouseStyle_2Story,RoofStyle_Hip,RoofMatl_Rare_val,Exterior1st_BrkFace,Exterior1st_CemntBd,Exterior1st_HdBoard,Exterior1st_Wd Sdng,Exterior2nd_Plywood,Exterior2nd_Stucco,Exterior2nd_VinylSd,MasVnrType_BrkFace,ExterQual_Gd,ExterQual_TA,Foundation_CBlock,Foundation_PConc,BsmtQual_Fa,BsmtQual_Gd,BsmtQual_TA,BsmtCond_Missing,BsmtCond_TA,BsmtExposure_Gd,BsmtExposure_Missing,BsmtExposure_No,BsmtFinType1_BLQ,BsmtFinType1_GLQ,BsmtFinType1_LwQ,BsmtFinType1_Unf,BsmtFinType2_Unf,HeatingQC_Gd,HeatingQC_TA,CentralAir_Y,KitchenQual_Fa,KitchenQual_Gd,KitchenQual_TA,Functional_Typ,GarageType_Missing,GarageType_Rare_val,GarageFinish_Missing,GarageFinish_RFn,GarageFinish_Unf,GarageQual_Missing,GarageQual_Rare_val,GarageCond_Missing,PavedDrive_Y,Fence_MnPrv,SaleType_New,SaleType_WD,SaleCondition_Normal
0,0.000000,0.428726,0.444444,0.625,0.360294,0.819672,0.0000,0.373438,0.000000,0.0,0.333333,0.0,0.250000,0.000000,0.457944,0.25,0.163361,0.25,0.454545,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,1.0
1,0.000000,0.468857,0.555556,0.625,0.382353,0.868852,0.0675,0.522632,0.000000,0.0,0.333333,0.5,0.333333,0.000000,0.485981,0.25,0.458576,0.00,0.454545,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0
2,0.235294,0.462769,0.444444,0.500,0.095588,0.213115,0.0000,0.386718,0.339467,0.0,0.666667,0.5,0.333333,0.333333,0.121495,0.50,0.247375,0.00,0.181818,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,1.0
3,0.235294,0.398875,0.555556,0.625,0.088235,0.213115,0.0125,0.385901,0.328329,0.0,0.666667,0.5,0.416667,0.333333,0.112150,0.50,0.420070,0.00,0.454545,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0
4,0.588235,0.263841,0.777778,0.500,0.132353,0.311475,0.0000,0.508416,0.000000,0.0,0.666667,0.0,0.250000,0.000000,0.168224,0.50,0.000000,0.30,0.000000,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0


(1459, 95)


In [118]:
X_test.isna().sum().sum()

0

In [119]:
train = pd.read_csv('train.csv')
y_train = train['SalePrice']
print(y_train.head())

0    208500
1    181500
2    223500
3    140000
4    250000
Name: SalePrice, dtype: int64


In [120]:
RMSEs = {}

#### Ridge

In [121]:
np.logspace(-1, 1, 1000)

array([ 0.1       ,  0.10046204,  0.10092622,  0.10139254,  0.10186102,
        0.10233166,  0.10280447,  0.10327947,  0.10375667,  0.10423607,
        0.10471768,  0.10520152,  0.1056876 ,  0.10617592,  0.1066665 ,
        0.10715934,  0.10765446,  0.10815187,  0.10865158,  0.10915359,
        0.10965793,  0.11016459,  0.1106736 ,  0.11118496,  0.11169868,
        0.11221478,  0.11273326,  0.11325413,  0.11377741,  0.11430311,
        0.11483124,  0.11536181,  0.11589483,  0.11643031,  0.11696827,
        0.11750871,  0.11805165,  0.1185971 ,  0.11914507,  0.11969557,
        0.12024861,  0.12080421,  0.12136238,  0.12192313,  0.12248646,
        0.1230524 ,  0.12362095,  0.12419214,  0.12476596,  0.12534243,
        0.12592156,  0.12650337,  0.12708787,  0.12767507,  0.12826498,
        0.12885762,  0.129453  ,  0.13005113,  0.13065202,  0.13125568,
        0.13186214,  0.1324714 ,  0.13308347,  0.13369837,  0.13431612,
        0.13493671,  0.13556018,  0.13618652,  0.13681576,  0.13

In [122]:
alphas = np.logspace(-1, 1, 1000)

ridgecv = RidgeCV(alphas = alphas, cv = 10)
ridgecv.fit(X_train, y_train)

print('Best Alpha Value from RidgeCV:', ridgecv.alpha_)

Best Alpha Value from RidgeCV: 2.1544346900318834


In [123]:
ridge = Ridge(alpha = ridgecv.alpha_)
ridge.fit(X_train, y_train)
y_pred = ridge.predict(X_train)
RMSEs['Ridge'] = np.sqrt(mean_squared_error(y_train, y_pred))
print(RMSEs['Ridge'])

26406.869688797884


#### Decision Tree

In [124]:
dt = DecisionTreeRegressor(random_state = 0)
dt.fit(X_train, y_train)
y_train_pred = dt.predict(X_train)
RMSEs['DT'] = np.sqrt(mean_squared_error(y_train, y_train_pred))
print(RMSEs['DT'])

0.0


#### Tuning Decision Tree

In [125]:
dt = DecisionTreeRegressor(random_state = 0)

params = {'max_depth' : sp_randint(2, 100),
         'min_samples_split' : sp_randint(2, 25),
          'min_samples_leaf' : sp_randint(1, 25)}

rsearch = RandomizedSearchCV(dt, param_distributions = params,
                            n_iter = 100, cv = 10, random_state = 0)
rsearch.fit(X_train, y_train)
print(rsearch.best_params_)

{'max_depth': 13, 'min_samples_leaf': 15, 'min_samples_split': 20}


##### Tuned Decision Tree

In [126]:
dt = DecisionTreeRegressor(**rsearch.best_params_, random_state = 0)
dt.fit(X_train, y_train)

y_train_pred = dt.predict(X_train)

RMSEs['DT'] = np.sqrt(mean_squared_error(y_train, y_train_pred))
print(RMSEs['DT'])

31979.03245721867


### Random Forest

In [127]:
rf = RandomForestRegressor(random_state = 0)

rf.fit(X_train, y_train)

y_train_pred = rf.predict(X_train)

RMSEs['RF'] = np.sqrt(mean_squared_error(y_train, y_train_pred))
print(RMSEs['RF'])

11281.165004868104


#### Tuning Random Forest

In [128]:
rf = RandomForestRegressor(random_state = 0)

params = {'n_estimators' : sp_randint(50, 200),
         'max_features' : sp_randint(1, 95),
         'max_depth' : sp_randint(2, 50),
         'min_samples_leaf' : sp_randint(1, 25),
         'min_samples_split' : sp_randint(2, 25),
         'oob_score' : [False, True]}

rsearch = RandomizedSearchCV(rf, param_distributions = params, 
                             n_iter = 150, cv = 10, random_state = 0)
rsearch.fit(X_train, y_train)
print(rsearch.best_params_)

{'max_depth': 30, 'max_features': 35, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 103, 'oob_score': True}


##### Tuned Random Forest

In [129]:
rf = RandomForestRegressor(**rsearch.best_params_, random_state = 0)

rf.fit(X_train, y_train)

y_train_pred = rf.predict(X_train)

RMSEs['RF'] = np.sqrt(mean_squared_error(y_train, y_train_pred))
print(RMSEs['RF'])

10745.33626169421


In [130]:
imp = pd.DataFrame(rf.feature_importances_, index = X_train.columns, columns = ['Importance'])
display(imp.sort_values(by = 'Importance', ascending = False))

,Importance
OverallQual,0.308258
GarageCars,0.106337
1stFlrSF,0.086605
YearBuilt,0.071057
ExterQual_TA,0.061099
...,...
Neighborhood_Sawyer,0.000169
BsmtQual_Fa,0.000160
Neighborhood_Mitchel,0.000138
GarageType_Rare_val,0.000134


### KNN

In [131]:
knn = KNeighborsRegressor()

knn.fit(X_train, y_train)

y_train_pred = knn.predict(X_train)

RMSEs['KNN'] = np.sqrt(mean_squared_error(y_train, y_train_pred))
print(RMSEs['KNN'])

32843.95882054941


#### Tuning KNN

In [132]:
knn = KNeighborsRegressor()
params = {'n_neighbors' : sp_randint(1, 25),
         'p' : sp_randint(1, 10)}

rsearch = RandomizedSearchCV(knn, param_distributions = params, 
                             cv = 10, n_iter = 25, 
                             random_state = 0)
rsearch.fit(X_train, y_train)
print(rsearch.best_params_)

{'n_neighbors': 10, 'p': 1}


##### Tuned KNN

In [133]:
knn = KNeighborsRegressor(**rsearch.best_params_)

knn.fit(X_train, y_train)

y_train_pred = knn.predict(X_train)

RMSEs['KNN'] = np.sqrt(mean_squared_error(y_train, y_train_pred))
print(RMSEs['KNN'])

35189.87401966457


### AdaBoost

In [134]:
ada = AdaBoostRegressor(random_state = 0)
ada.fit(X_train, y_train)

y_train_pred = ada.predict(X_train)

RMSEs['Ada'] = np.sqrt(mean_squared_error(y_train, y_train_pred))
print(RMSEs['Ada'])

30560.100156733362


#### Tuning AdaBoost

In [140]:
ada = AdaBoostRegressor(random_state = 0)
params = {'n_estimators' : sp_randint(25, 100),
         'learning_rate' : np.logspace(0, 0.25, 100),
         'loss' : ['linear', 'square']}

rsearch = RandomizedSearchCV(ada, param_distributions = params, 
                             cv = 10, n_iter = 100, 
                             random_state = 0)
rsearch.fit(X_train, y_train)
print(rsearch.best_params_)

{'learning_rate': 1.0294997906179981, 'loss': 'linear', 'n_estimators': 42}


##### Tuned AdaBoost

In [142]:
ada = AdaBoostRegressor(**rsearch.best_params_, random_state = 0)

ada.fit(X_train, y_train)

y_train_pred = ada.predict(X_train)

RMSEs['Ada'] = np.sqrt(mean_squared_error(y_train, y_train_pred))
print(RMSEs['Ada'])

31331.950847285876


### LightGBM

In [143]:
lgbr = lgb.LGBMRegressor(random_state = 0)

lgbr.fit(X_train, y_train)

y_train_pred = lgbr.predict(X_train)

RMSEs['LGBM'] = np.sqrt(mean_squared_error(y_train, y_train_pred))
print(RMSEs['LGBM'])

13081.172890871045


#### Tuning LightGBM

In [149]:
np.logspace(-1, 0.5, 150)

array([0.1       , 0.10234511, 0.10474522, 0.10720162, 0.10971562,
       0.11228858, 0.11492187, 0.11761692, 0.12037517, 0.1231981 ,
       0.12608724, 0.12904413, 0.13207036, 0.13516756, 0.1383374 ,
       0.14158157, 0.14490182, 0.14829993, 0.15177773, 0.15533709,
       0.15897992, 0.16270818, 0.16652388, 0.17042905, 0.17442581,
       0.17851629, 0.1827027 , 0.18698729, 0.19137235, 0.19586025,
       0.2004534 , 0.20515426, 0.20996536, 0.21488929, 0.21992869,
       0.22508626, 0.23036479, 0.23576711, 0.24129612, 0.24695479,
       0.25274616, 0.25867334, 0.26473953, 0.27094797, 0.27730201,
       0.28380506, 0.29046061, 0.29727224, 0.30424362, 0.31137848,
       0.31868066, 0.32615408, 0.33380277, 0.34163082, 0.34964246,
       0.35784197, 0.36623377, 0.37482237, 0.38361238, 0.39260853,
       0.40181565, 0.41123868, 0.4208827 , 0.43075288, 0.44085452,
       0.45119307, 0.46177406, 0.47260319, 0.48368627, 0.49502926,
       0.50663826, 0.51851951, 0.53067938, 0.54312442, 0.55586

In [148]:
lgbr = lgb.LGBMRegressor(random_state = 0)

params = {'n_estimators' : sp_randint(50, 200),
         'max_depth' : sp_randint(1, 25),
         'learning_rate' : np.logspace(-1, 0.5, 150)}

rsearch_lgbm = RandomizedSearchCV(lgbr, param_distributions = params,
                                 cv = 10, n_iter = 150, random_state = 0)

rsearch_lgbm.fit(X_train, y_train)
print(rsearch_lgbm.best_params_)

{'learning_rate': 0.21992868586060285, 'max_depth': 3, 'n_estimators': 138}


##### Tuned LightGBM

In [150]:
lgbr = lgb.LGBMRegressor(**rsearch_lgbm.best_params_, random_state = 0)

lgbr.fit(X_train, y_train)

y_train_pred = lgbr.predict(X_train)

RMSEs['LGBM'] = np.sqrt(mean_squared_error(y_train, y_train_pred))
print(RMSEs['LGBM'])

15191.039788656703


### Stacking

In [151]:
print(RMSEs)

{'Ridge': 26406.869688797884, 'DT': 31979.03245721867, 'RF': 10745.33626169421, 'KNN': 35189.87401966457, 'Ada': 31331.950847285876, 'LGBM': 15191.039788656703}


#### Voting Classifier

In [155]:
votreg = VotingRegressor(estimators = [('rf', rf), ('lgbr', lgbr)])

votreg.fit(X_train, y_train)

y_train_pred = votreg.predict(X_train)
print("Voting Regressor RMSE:", np.sqrt(mean_squared_error(y_train, y_train_pred)))

Voting Regressor RMSE: 12058.216475921829


#### Preparing the data for submission

In [99]:
test = pd.read_csv('test.csv')
display(test.head())

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,HeatingQC,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
0,1461,20,RH,80.0,11622,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,NAmes,Feedr,Norm,1Fam,1Story,5,6,1961,1961,Gable,CompShg,VinylSd,VinylSd,None,0.0,TA,TA,CBlock,TA,TA,No,Rec,468.0,LwQ,144.0,270.0,882.0,GasA,TA,Y,SBrkr,896,0,0,896,0.0,0.0,1,0,2,1,TA,5,Typ,0,NaN,Attchd,1961.0,Unf,1.0,730.0,TA,TA,Y,140,0,0,0,120,0,NaN,MnPrv,NaN,0,6,2010,WD,Normal
1,1462,20,RL,81.0,14267,Pave,NaN,IR1,Lvl,AllPub,Corner,Gtl,NAmes,Norm,Norm,1Fam,1Story,6,6,1958,1958,Hip,CompShg,Wd Sdng,Wd Sdng,BrkFace,108.0,TA,TA,CBlock,TA,TA,No,ALQ,923.0,Unf,0.0,406.0,1329.0,GasA,TA,Y,SBrkr,1329,0,0,1329,0.0,0.0,1,1,3,1,Gd,6,Typ,0,NaN,Attchd,1958.0,Unf,1.0,312.0,TA,TA,Y,393,36,0,0,0,0,NaN,NaN,Gar2,12500,6,2010,WD,Normal
2,1463,60,RL,74.0,13830,Pave,NaN,IR1,Lvl,AllPub,Inside,Gtl,Gilbert,Norm,Norm,1Fam,2Story,5,5,1997,1998,Gable,CompShg,VinylSd,VinylSd,None,0.0,TA,TA,PConc,Gd,TA,No,GLQ,791.0,Unf,0.0,137.0,928.0,GasA,Gd,Y,SBrkr,928,701,0,1629,0.0,0.0,2,1,3,1,TA,6,Typ,1,TA,Attchd,1997.0,Fin,2.0,482.0,TA,TA,Y,212,34,0,0,0,0,NaN,MnPrv,NaN,0,3,2010,WD,Normal
3,1464,60,RL,78.0,9978,Pave,NaN,IR1,Lvl,AllPub,Inside,Gtl,Gilbert,Norm,Norm,1Fam,2Story,6,6,1998,1998,Gable,CompShg,VinylSd,VinylSd,BrkFace,20.0,TA,TA,PConc,TA,TA,No,GLQ,602.0,Unf,0.0,324.0,926.0,GasA,Ex,Y,SBrkr,926,678,0,1604,0.0,0.0,2,1,3,1,Gd,7,Typ,1,Gd,Attchd,1998.0,Fin,2.0,470.0,TA,TA,Y,360,36,0,0,0,0,NaN,NaN,NaN,0,6,2010,WD,Normal
4,1465,120,RL,43.0,5005,Pave,NaN,IR1,HLS,AllPub,Inside,Gtl,StoneBr,Norm,Norm,TwnhsE,1Story,8,5,1992,1992,Gable,CompShg,HdBoard,HdBoard,None,0.0,Gd,TA,PConc,Gd,TA,No,ALQ,263.0,Unf,0.0,1017.0,1280.0,GasA,Ex,Y,SBrkr,1280,0,0,1280,0.0,0.0,2,0,2,1,Gd,5,Typ,0,NaN,Attchd,1992.0,RFn,2.0,506.0,TA,TA,Y,0,82,0,0,144,0,NaN,NaN,NaN,0,1,2010,WD,Normal


In [153]:
X_test.isna().sum().sum()

0

In [157]:
y_test_pred = votreg.predict(X_test)
solution = pd.DataFrame({'Id' : test['Id'], 'SalePrice' : y_test_pred})

# Export to CSV
solution.to_csv("Solution3_House_Dum_VotRfLgbr_Saurabh.csv", index = False)

solution.head()

,Id,SalePrice
0,1461,119374.858353
1,1462,149081.791889
2,1463,177866.348424
3,1464,181271.341449
4,1465,196358.907926
